In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import numpy as np

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
MULTIPLI_PARCERIAS = 6

## Abrindo planilha parcerias Marian

In [ ]:
parcerias = pd.read_excel('original_data/Cópia de parcerias122020-2.xlsx')

In [ ]:
parcerias.sample(5)

In [ ]:
parcerias.info()

Filtro situacao parcerias aprovada

In [ ]:
parcerias['situacao'].unique()

In [ ]:
filtro_aprovados = parcerias['situacao']=='Aprovado'

In [ ]:
filtro_aprovados

In [ ]:
len(parcerias)

In [ ]:
parcerias = parcerias[filtro_aprovados].copy().reset_index(drop=True)

In [ ]:
parcerias.sample(5)

In [ ]:
parcerias['valor_mensal'].isnull().sum()

In [ ]:
filtro_valor_mensal = parcerias['valor_mensal'].isnull()

In [ ]:
parcerias = parcerias[~filtro_valor_mensal].copy().reset_index(drop=True)

In [ ]:
parcerias['valor_mensal'].isnull().sum()

In [ ]:
colunas = ['codigo_escola', 'valor_mensal']

In [ ]:
parcerias = parcerias[colunas].copy()

In [ ]:
parcerias

In [ ]:
parcerias.info()

Pegar distritos das escolas

In [ ]:
escolas = pd.read_excel('original_data/endereços_escolas.xlsx')

In [ ]:
escolas.info()

In [ ]:
escolas.sample(5)

In [ ]:
colunas = ['distrito', 'codesc']

In [ ]:
escolas = escolas[colunas].copy()

Join escolas

In [ ]:
escolas.keys()

In [ ]:
len(escolas)

In [ ]:
len(parcerias)

In [ ]:
parcerias.keys()

In [ ]:
parcerias.dtypes

In [ ]:
escolas.dtypes

In [ ]:
escolas.keys()

In [ ]:
def arrumar_digitos(item):
    
    item = str(item)
    
    while len(item)<6:
        item = '0' + item
    
    return item

In [ ]:
escolas['codesc'] = escolas['codesc'].apply(arrumar_digitos)

In [ ]:
parcerias['codigo_escola'] = parcerias['codigo_escola'].astype(int)

In [ ]:
parcerias['codigo_escola' ] = parcerias['codigo_escola'].apply(arrumar_digitos)

In [ ]:
parcerias_distrito = pd.merge(parcerias, escolas, left_on='codigo_escola', right_on = 'codesc', how='left')

In [ ]:
parcerias_distrito[parcerias_distrito['distrito'].isnull()]

In [ ]:
parcerias_distrito = parcerias_distrito[parcerias_distrito['distrito'].notnull()].copy()

In [ ]:
parcerias_distrito['distrito'].isnull().sum()

In [ ]:
len(parcerias_distrito) == len(parcerias)-2

In [ ]:
parcerias_distrito['valor_total'] = parcerias_distrito['valor_mensal']* MULTIPLI_PARCERIAS

JOIN PLANILHA HOMERO

In [ ]:
pagamentos = pd.read_csv('original_data/Folha PMSP 1º semestre 2020 vf.csv', sep=';')

In [ ]:
pagamentos.info()

In [ ]:
colunas = ['ORGAO', 'REMUNERACAO BRUTA', 'DISTRITO']

In [ ]:
pagamentos = pagamentos[colunas].copy()

In [ ]:
pagamentos[pagamentos['ORGAO'].str.contains('EDU')]['ORGAO'].unique()

In [ ]:
sorted(pagamentos['ORGAO'].unique())

In [ ]:
filtro_educacao = pagamentos['ORGAO']=='SECRETARIA MUNICIPAL DE EDUCACAO'

In [ ]:
pagamentos = pagamentos[filtro_educacao].reset_index(drop=True).copy()

In [ ]:
sorted(pagamentos['DISTRITO'].unique())

In [ ]:
len(pagamentos['DISTRITO'].unique()) <= 96

In [ ]:
pagamentos.drop('ORGAO', axis =1, inplace=True)

In [ ]:
pagamentos.dtypes

In [ ]:
#pagamentos['REMUNERACAO BRUTA'].astype(float)

In [ ]:
def arrumar_pagamento(item):
    
    if pd.isnull(item):
        return np.NaN
    
    item = str(item)
    item = item.replace('.', '')
    item = item.replace(',', '.')

    
    return float(item)
    

In [ ]:
arrumar_pagamento('22.574,98')

In [ ]:
pagamentos['REMUNERACAO BRUTA'] = pagamentos['REMUNERACAO BRUTA'].apply(arrumar_pagamento)

In [ ]:
agrupado_pagamentos = pagamentos.groupby('DISTRITO').sum()

In [ ]:
agrupado_pagamentos

In [ ]:
agrupado_pagamentos = agrupado_pagamentos.reset_index()

In [ ]:
parcerias_distrito.dtypes

In [ ]:
agrupado_parcerias = parcerias_distrito.groupby('distrito').sum()[['valor_total']]

In [ ]:
agrupado_parcerias = agrupado_parcerias.reset_index()

In [ ]:
agrupado_parcerias

In [ ]:
set(agrupado_parcerias['distrito'])-set(agrupado_pagamentos['DISTRITO'])

In [ ]:
set(agrupado_pagamentos['DISTRITO'])-set(agrupado_parcerias['distrito'])

In [ ]:
df_final = pd.merge(agrupado_parcerias, agrupado_pagamentos, how='outer',
                    left_on = 'distrito', right_on = 'DISTRITO')

In [ ]:
df_final['valor_total'] = df_final['valor_total'].fillna(0)

In [ ]:
df_final['REMUNERACAO BRUTA'] = df_final['REMUNERACAO BRUTA'].fillna(0)

In [ ]:
df_final['total_gasto'] = df_final['REMUNERACAO BRUTA'] + df_final['valor_total']

In [ ]:
def resolver_distrito(row):
    
    if pd.isnull(row['distrito']):
        return row['DISTRITO']
    if pd.isnull(row['DISTRITO']):
        return row['distrito']
    
    return row['distrito']

In [ ]:
df_final['distrito_final'] = df_final.apply(resolver_distrito, axis=1)

In [ ]:
df_final = df_final.rename({'valor_total':'total_parcerias',
                 'REMUNERACAO BRUTA' : 'total_servidores'},
                axis=1)

In [ ]:
df_final = df_final.drop(['distrito', 'DISTRITO'], axis=1)

In [ ]:
df_final.to_excel('despesas_regionalizadas_educacao.xlsx')

In [ ]:
shape_distritos = gpd.read_file('original_data/SIRGAS_SHP_distrito/SIRGAS_SHP_distrito_polygon.shp')

In [ ]:
shape_distritos = shape_distritos.set_crs(epsg = 31983)

In [ ]:
df_final

In [ ]:
set(shape_distritos['ds_nome'])-set(df_final['distrito_final'])

In [ ]:
set(df_final['distrito_final']) - set(shape_distritos['ds_nome'])

In [ ]:
set(df_final['distrito_final']) == set(shape_distritos['ds_nome'])

In [ ]:
df_final_dists = pd.merge(df_final, shape_distritos, left_on ='distrito_final',
                         right_on='ds_nome')

In [ ]:
df_final_dists = gpd.GeoDataFrame(
    df_final_dists, geometry='geometry')

In [ ]:
df_final_dists = df_final_dists.set_crs(epsg = 31983)

In [ ]:
type(df_final_dists)

In [ ]:
df_final_dists['total_gasto_resum'] = df_final_dists['total_gasto']/1000000

In [ ]:
def plot_map(df, var, title, fname=None):
    import matplotlib.pyplot as plt
    import os
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(8.3, 11.7)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.axis('off')
    df.plot(column=var,
            ax=ax,
            cmap='GnBu',
            legend=True,
            legend_kwds={
                'label': f"{var}",
                'format': "%.0f milhões"},
           edgecolor='black')
    plt.title(title)
    
    if fname is not None:
        plt.savefig(os.path.abspath(os.path.join(
            os.getcwd(), fname)))
    
    return fig

In [ ]:
fig = plot_map(df_final_dists, 'total_gasto_resum', 'Regionalização Gastos Educação',
              'gasto_educacao_regionalizado.jpeg')